In [4]:
import pandas as pd

df: pd.DataFrame = pd.read_csv('../data/transactions.csv')
df_parquet: pd.DataFrame = pd.read_parquet('../data/transactions.parquet')

In [5]:
from tqdm import tqdm

index_to_address = {}
address_to_index = {}

def compress_df(df: pd.DataFrame):
    """
    Replaces 'from_address' and 'to_address' with a unique integer,
    reducing the size of the dataframe.
    """
    pbar = tqdm(total=len(df))
    counter = 0
    for index, row in df.iterrows():
        if address_to_index.get(row['from_address']) is None and address_to_index.get(row['from_address']) == address_to_index.get(row['to_address']):
            index_to_address[counter] = row['from_address']
            address_to_index[row['from_address']] = counter
            index_to_address[counter] = row['to_address']
            address_to_index[row['to_address']] = counter
            counter += 1
        else:
            if address_to_index.get(row['from_address']) is None:
                index_to_address[counter] = row['from_address']
                address_to_index[row['from_address']] = counter
                counter += 1
            if address_to_index.get(row['to_address']) is None:
                index_to_address[counter] = row['to_address']
                address_to_index[row['to_address']] = counter
                counter += 1
        df.at[index, 'from_address'] = address_to_index[row['from_address']]
        df.at[index, 'to_address'] = address_to_index[row['to_address']]
        pbar.update()

def decompress_df(df: pd.DataFrame):
    """
    Replaces unique integer with 'from_address' and 'to_address'.
    """
    for index, row in df.iterrows():
        df.at[index, 'from_address'] = index_to_address[row['from_address']]
        df.at[index, 'to_address'] = index_to_address[row['to_address']]


compress_df(df)
compress_df(df_parquet)
#df.to_csv('../data/transactions.csv', index=False,header=True)

100%|██████████| 5103881/5103881 [03:25<00:00, 24813.31it/s]
